# Statistical Analysis of Basic Baskets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from classes.datacleaner import DataCleaner

In [3]:
df = pd.read_excel('data/ICB_2s-2025.xlsx')
df.head()

,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK
0,2022-09-12,SUP-0,Leite,Parmalat,6.28,1.0,6.280
1,2022-09-12,SUP-1,Tomate,NaN,6.99,1.0,6.990
2,2022-09-12,SUP-2,Arroz,Namorado,17.98,5.0,3.596
3,2022-09-12,SUP-2,Arroz,Broto Legal,22.69,5.0,4.538
4,2022-09-12,SUP-0,Feijão,Broto Legal,10.98,1.0,NaN


## *Data Overview*

In [4]:
# Display basic information about the dataset
print("Dataset Shape:", df.shape)

print("\nMissing Values:")
df.isnull().sum()

Dataset Shape: (34032, 7)

Missing Values:


Data_Coleta           0
Estabelecimento       0
Produto               0
Marca              6395
Preco                 0
Quantidade            0
PPK                   1
dtype: int64

In [5]:
numerical_cols = df.select_dtypes(include=[np.number]).columns

print(f"\nNumerical Columns: {len(numerical_cols)}")
print(f"Categorical Columns: {len(df.select_dtypes(include=['object']).columns)}")

print("\nData Types:")
df.dtypes



Numerical Columns: 3
Categorical Columns: 3

Data Types:


Data_Coleta        datetime64[ns]
Estabelecimento            object
Produto                    object
Marca                      object
Preco                     float64
Quantidade                float64
PPK                       float64
dtype: object

## *Exploration & Cleanup*

In [6]:
df.groupby('Produto')

### Combination of Product Classes

In [7]:
# Function to list unique products
def list_products(df):
    products = df['Produto'].unique()
    print(f"\nProdutos encontrados:")
    for i, product in enumerate(products, 1):
        print(f"{i}. {product}")

In [8]:
# Count total distinct products
total_distinct_products = df['Produto'].unique()
print(f"Total de produtos distintos: {len(total_distinct_products)}")

# Show all distinct products
list_products(df)

Total de produtos distintos: 27

Produtos encontrados:
1. Leite
2. Tomate
3. Arroz
4. Feijão
5. Açúcar
6. Manteiga
7. Óleo
8. Carne Coxão Mole
9. Café
10. Banana Prata
11. Farinha
12. Batata
13. Banana Nanica
14. Pão
15. Frango Peito
16. Macarrão
17. Carne Pernil
18. Ovo
19. Carne Acém
20. Frango Sobrecoxa
21. Carne Acem
22. Farinha de Trigo
23. Macarrão com Ovos
24. Pão Francês
25. Carne Bovina Acem
26. Carne Bovina Coxão Mole
27. Carne Suína Pernil


#### Carne Bovina Acém
Replacing Acem variations into a single type

In [9]:
mapping = {k: "Carne Bovina Acém" for k in ['Carne Acém', 'Carne Acem', 'Carne Bovina Acem']}
df['Produto'] = df['Produto'].replace(mapping)

distinct_products = df['Produto'].unique()
print(f"Novo total de produtos distintos: {len(distinct_products)}")

Novo total de produtos distintos: 25


#### Carne Bovina Coxão Mole
Replacing Carne Coxão Mole variation into a single type

In [10]:
df['Produto'] = df['Produto'].replace({'Carne Coxão Mole': 'Carne Bovina Coxão Mole'})

#### Pão
Analysis and replacement of 'Pão Francês' to 'Pão'

In [11]:
pao = df[df['Produto'] == 'Pão']
pao_frances = df[df['Produto'] == 'Pão Francês']

In [12]:
# Comparative statistical summary: Pão vs Pão Francês
cols = ['Preco']

def summarize(s):
    s = s.dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    return {
        'count': int(s.count()),
        'mean': float(s.mean()),
        'median': float(s.median()),
        'std': float(s.std()),
        'min': float(s.min()),
        'max': float(s.max()),
        'q1': float(q1),
        'q3': float(q3),
        'iqr': float(q3 - q1)
    }

results = {}
for col in cols:
    s1 = pao[col]
    s2 = pao_frances[col]
    results[col] = {
        'pao': summarize(s1),
        'pao_frances': summarize(s2),
        'mean_diff': float(s1.mean() - s2.mean()),
    }

# Print concise table-like summary
for col, res in results.items():
    print(f"--- {col} ---")
    print("Pão:      ", res['pao'])
    print("Pão Francês:", res['pao_frances'])
    print(f"Mean difference (Pão - Pão Francês): {res['mean_diff']:.4f}")
    print()

--- Preco ---
Pão:       {'count': 783, 'mean': 16.290217113665392, 'median': 15.99, 'std': 1.7672672645966512, 'min': 3.9, 'max': 26.5, 'q1': 15.79, 'q3': 16.99, 'iqr': 1.1999999999999993}
Pão Francês: {'count': 23, 'mean': 17.819130434782608, 'median': 17.99, 'std': 1.5293370906241999, 'min': 14.9, 'max': 19.99, 'q1': 16.924999999999997, 'q3': 18.945, 'iqr': 2.020000000000003}
Mean difference (Pão - Pão Francês): -1.5289



>Given the minimal diferences saw, we could infer them as being equal. As well, since 'Pão Francês' has little samples at all, it would not be meaningful to keep.

In [13]:
df['Produto'] = df['Produto'].replace({'Pão Francês': 'Pão'})

#### Farinha de Trigo
Analysis and replacement of 'Farinha' into 'Farinha de Trigo'

In [14]:
farinha = df[df['Produto'] == 'Farinha']
farinha_trigo = df[df['Produto'] == 'Farinha de Trigo']

In [15]:
results = {}
for col in cols:
    s1 = farinha[col]
    s2 = farinha_trigo[col]
    results[col] = {
        'farinha': summarize(s1),
        'farinha_trigo': summarize(s2),
        'mean_diff': float(s1.mean() - s2.mean()),
    }

# Print concise table-like summary
for col, res in results.items():
    print(f"--- {col} ---")
    print("Farinha:      ", res['farinha'])
    print("Farinha de Trigo:", res['farinha_trigo'])
    print(f"Mean difference (Farinha - Farinha de Trigo): {res['mean_diff']:.4f}")
    print()

--- Preco ---
Farinha:       {'count': 3199, 'mean': 6.033514535792435, 'median': 5.99, 'std': 1.3034269372786749, 'min': 2.49, 'max': 13.99, 'q1': 5.09, 'q3': 6.99, 'iqr': 1.9000000000000004}
Farinha de Trigo: {'count': 72, 'mean': 5.797499999999999, 'median': 5.59, 'std': 1.3116903941635178, 'min': 2.99, 'max': 9.99, 'q1': 4.99, 'q3': 6.49, 'iqr': 1.5}
Mean difference (Farinha - Farinha de Trigo): 0.2360



>Given the minimal diferences saw, we could infer them as being equal. We will keep the products as 'Farinha de Trigo' due to its more representative quality too.

In [16]:
df['Produto'] = df['Produto'].replace({'Farinha': 'Farinha de Trigo'})

#### Macarrão
We see the same pattern occurs between 'Macarrão' vs 'Macarrão com Ovos', so we will joint those into 'Macarrão'

In [17]:
df['Produto'] = df['Produto'].replace({'Macarrão com Ovos': 'Macarrão'})

#### Carne Suína Pernil
As found in Acem problem, we have a type of meat being replicated here, so we will joint the two types into the more representative "Carne Suína Pernil" label

In [18]:
df['Produto'] = df['Produto'].replace({'Carnel Pernil': 'Carne Suína Pernil'})

#### Final Results (New Product List)

In [19]:
print(f"Number of merged products classes: {len(total_distinct_products) - len(df['Produto'].unique())}")
print("\nUpdated Product List:")
list_products(df)

Number of merged products classes: 6

Updated Product List:

Produtos encontrados:
1. Leite
2. Tomate
3. Arroz
4. Feijão
5. Açúcar
6. Manteiga
7. Óleo
8. Carne Bovina Coxão Mole
9. Café
10. Banana Prata
11. Farinha de Trigo
12. Batata
13. Banana Nanica
14. Pão
15. Frango Peito
16. Macarrão
17. Carne Pernil
18. Ovo
19. Carne Bovina Acém
20. Frango Sobrecoxa
21. Carne Suína Pernil


### Products Categorization
For even more easily analysis, we will leave a formal dict with products distinguished by general classes

In [20]:
# Addressing Products to classes
product_classes = {
    'Vegetais': ['Tomate', 'Banana Prata', 'Banana Nanica', 'Batata'],
    'Carnes Vermelhas': ['Carne Bovina Acém', 'Carne Bovina Coxão Mole', 
               'Carne Suína Pernil'],
    'Aves': ['Frango Peito', 'Frango Sobrecoxa'],
    'Laticínios': ['Manteiga', 'Leite'],
    'Padaria & Cozinha': ['Pão', 'Ovo', 'Farinha de Trigo', 'Café', 'Açúcar', 'Óleo'],
    'Grãos & Massas': ['Arroz', 'Feijão', 'Macarrão'],
}
# invert product_classes into product -> class mapping
prod_to_class = {prod: cls for cls, prods in product_classes.items() for prod in prods}

# dataframe with each distinct product and its class
class_df = pd.DataFrame({'Produto': df['Produto'].unique()})
class_df['Classe'] = class_df['Produto'].map(prod_to_class).fillna('Unassigned')
class_df = class_df.sort_values('Classe').reset_index(drop=True)

class_df



,Produto,Classe
0,Frango Peito,Aves
1,Frango Sobrecoxa,Aves
2,Carne Suína Pernil,Carnes Vermelhas
3,Carne Bovina Acém,Carnes Vermelhas
4,Carne Bovina Coxão Mole,Carnes Vermelhas
5,Arroz,Grãos & Massas
6,Feijão,Grãos & Massas
7,Macarrão,Grãos & Massas
8,Leite,Laticínios
9,Manteiga,Laticínios


In [21]:
# Adicionando a coluna 'Classe' ao DataFrame principal usando o mapeamento
df['Classe'] = df['Produto'].map(prod_to_class)

### Brand Analysis

#### Products without brand
To ensure consistency, we will assign the **known product classes with no brand** to the value 'Sem Marca'

In [22]:
# Product classes without 'Marca': all meat types, all fruit types, eggs, bread
without_brand = ['Tomate', 'Banana Prata', 'Banana Nanica', 'Batata',
                 'Pão', 'Frango Peito', 'Ovo', 'Carne Bovina Acém', 
                 'Frango Sobrecoxa', 'Carne Bovina Coxão Mole', 'Carne Suína Pernil']
print(f'Quantity of product classes without brand attached to: {len(without_brand)}')

# Assign 'Sem Marca' to products without brand
df = df.copy()
df.loc[df['Produto'].isin(without_brand), 'Marca'] = 'Sem Marca'
df[df['Produto'].isin(without_brand)].head()

Quantity of product classes without brand attached to: 11


,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK,Classe
1,2022-09-12,SUP-1,Tomate,Sem Marca,6.99,1.0,6.99,Vegetais
11,2022-09-12,SUP-0,Carne Bovina Coxão Mole,Sem Marca,42.90,1.0,42.90,Carnes Vermelhas
13,2022-09-12,SUP-4,Carne Bovina Coxão Mole,Sem Marca,42.93,1.0,42.93,Carnes Vermelhas
20,2022-09-12,SUP-0,Tomate,Sem Marca,3.75,1.0,3.75,Vegetais
21,2022-09-12,SUP-1,Banana Prata,Sem Marca,9.90,1.0,9.90,Vegetais


#### Products without brand which should have one

In [23]:
# Count null values for each 'Produto' class
null_counts_by_product = df.groupby('Produto')['Marca'].apply(lambda x: x.isnull().sum())
null_summary_table = pd.DataFrame({
    'Produto': null_counts_by_product.index,
    'Null_Marca_Count': null_counts_by_product.values,
    'Total_Records': df.groupby('Produto').size().values,
    'Null_Percentage': (null_counts_by_product.values / df.groupby('Produto').size().values * 100).round(2)
})

null_summary_table = null_summary_table.sort_values('Null_Marca_Count', ascending=False)
null_summary_table


,Produto,Null_Marca_Count,Total_Records,Null_Percentage
8,Carne Pernil,154,154,100.00
14,Leite,8,3490,0.23
11,Feijão,5,3602,0.14
5,Café,4,4122,0.10
0,Arroz,4,4015,0.10
10,Farinha de Trigo,3,3271,0.09
16,Manteiga,2,3645,0.05
20,Óleo,1,2466,0.04
1,Açúcar,1,2420,0.04
4,Batata,0,1018,0.00


> Due to the notiable small ammount of null values, we can securely discart those...

In [24]:
# Remotion of null values in 'Marca' column
df = df.dropna(subset=['Marca'])

#### Brands incorrectly assigned
The dataset have several mistypes on brand assignation of products, so we need to fix those

In [25]:
# Convert all brands to string type to handle mixed data types
df['Marca'] = df['Marca'].astype(str)

# Get unique brands and sort alphabetically
unique_brands = pd.DataFrame(df['Marca'].unique())
unique_brands = unique_brands.sort_values(by=0).reset_index(drop=True)

print(f'Total unique brands: {len(unique_brands)}')

unique_brands_array = unique_brands.values.flatten()
unique_brands_array


Total unique brands: 168


array(['3 Corações', '3 corações', 'Adria', 'Albaruska', 'Albaruska ',
       'Alto Alegre', 'Alto Alegre ', 'Anaconda', 'Aro', 'Aurora',
       'Aviação', 'Batavo', 'Bela Nata', 'Belarina', 'Biguá', 'Biju',
       'Bonolat', 'Brasileirissima', 'Broto Legal', 'Buque', 'Caboclo',
       'Caldo Bom', 'Caldo Nobre', 'Camil', 'Camil ', 'Canecão',
       'Caravelas', 'Carrefour', 'Catupiry', 'Celina', 'Claybom', 'Coamo',
       'Cocamar', 'Colombo', 'Concórdia', 'Corina', 'Cruzilia', 'DIA',
       'Da Barra', 'Da Casa', 'Da Vaca', 'DaVaca', 'Danone', 'Davaca',
       'Delícia', 'Dia', 'Dona Benta', 'Doriana', 'Dualis', 'Elegê',
       'Emporio São João', 'Empório São João', 'Famiglia Venturelli',
       'Fantástico', 'Fort', 'Frimesa', 'Galo', 'Gaspar', 'Globo',
       'Grão De Campo', 'Grão Do Campo', 'Grão de campo',
       'Grão de campo ', 'Grão do Campo', 'Guarani', 'Guarani ', 'Inari',
       'Italac', 'Itambé', 'Joli', 'Jussara', 'Kicaldo', 'Knor', 'Knorr',
       'Leco', 'Leve', 'Li

In [26]:
df['Marca'] = df['Marca'].replace({'3 corações': '3 Corações'})
df['Marca'] = df['Marca'].replace({'Albaruska ': 'Albaruska'})
df['Marca'] = df['Marca'].replace({'Alto Alegre ': 'Alto Alegre'})
df['Marca'] = df['Marca'].replace({'Camil ': 'Camil'})
df['Marca'] = df['Marca'].replace({'DaVaca': 'Da Vaca'})
df['Marca'] = df['Marca'].replace({'Davaca': 'Da Vaca'})
df['Marca'] = df['Marca'].replace({'Emporio São João': 'Empório São João'})
df['Marca'] = df['Marca'].replace({'Grão De Campo': 'Grão Do Campo'})
df['Marca'] = df['Marca'].replace({'Grão de Campo': 'Grão Do Campo'})
df['Marca'] = df['Marca'].replace({'Grão de Campo ': 'Grão Do Campo'})
df['Marca'] = df['Marca'].replace({'Grão do Campo': 'Grão Do Campo'})
df['Marca'] = df['Marca'].replace({'Guarani ': 'Guarani'})
df['Marca'] = df['Marca'].replace({'Knor': 'Knorr'})
df['Marca'] = df['Marca'].replace({'Lider': 'Líder'})
df['Marca'] = df['Marca'].replace({'Outra': 'Outro'})
df['Marca'] = df['Marca'].replace({'Outro2': 'Outro'})
df['Marca'] = df['Marca'].replace({'Pateko': 'Patéko'})
df['Marca'] = df['Marca'].replace({'Paulista ': 'Paulista'})
df['Marca'] = df['Marca'].replace({'Piracanjuba ': 'Piracanjuba'})
df['Marca'] = df['Marca'].replace({'Prato Fino ': 'Prato Fino'})
df['Marca'] = df['Marca'].replace({'Qualita': 'Qualitá'})
df['Marca'] = df['Marca'].replace({'Renata ': 'Renata'})
df['Marca'] = df['Marca'].replace({'Saboroso ': 'Saboroso'})
df['Marca'] = df['Marca'].replace({'Serrazul\n': 'Serrazul'})
df['Marca'] = df['Marca'].replace({'São': 'São Joaquim'})
df['Marca'] = df['Marca'].replace({'São Joaquim ': 'São Joaquim'})
df['Marca'] = df['Marca'].replace({'Urbano ': 'Urbano'})
df['Marca'] = df['Marca'].replace({'Vasconcelos ': 'Vasconcelos'})

print(f'New total of brands after cleaning: {len(df["Marca"].unique())}')

New total of brands after cleaning: 142


##### Sub-analysis: 'Outros' value assignation problemn
There is a considerable amount of records without the possibility to be corrected assigned

In [27]:
# Count 'Outro' values for each 'Produto' class (strip and lowercase to catch variants)
outro_counts_by_product = df.groupby('Produto')['Marca'].apply(lambda x: x.str.strip().str.lower().eq('outro').sum())
total_by_product = df.groupby('Produto').size()

outro_summary_table = pd.DataFrame({
    'Produto': outro_counts_by_product.index,
    'Outro_Marca_Count': outro_counts_by_product.values,
    'Total_Records': total_by_product.values,
    'Outro_Percentage': (outro_counts_by_product.values / total_by_product.values * 100).round(2)
})

print(f'Total of Outro brand entries: {outro_summary_table["Outro_Marca_Count"].sum()}. ({(outro_summary_table["Outro_Marca_Count"].sum() / len(df) * 100).round(2)}% of all records)')

outro_summary_table = outro_summary_table.sort_values('Outro_Marca_Count', ascending=False)
outro_summary_table


Total of Outro brand entries: 643. (1.9% of all records)


,Produto,Outro_Marca_Count,Total_Records,Outro_Percentage
15,Manteiga,168,3643,4.61
0,Arroz,93,4011,2.32
9,Farinha de Trigo,85,3268,2.60
10,Feijão,83,3597,2.31
13,Leite,71,3482,2.04
1,Açúcar,47,2419,1.94
14,Macarrão,39,472,8.26
19,Óleo,35,2465,1.42
5,Café,22,4118,0.53
2,Banana Nanica,0,910,0.00


> <span style="color:red">**WAITING REVIEW**</span> <br>
required to see what we will do with those products...

#### Brands review by Product
Analysis to look statistical information of each product type, based on it's brand

In [28]:
# Function for brand statistical review by product
def analyze_brand_by_product(df, product_name, price_col='Preco', iqr_threshold=1.5):
    """
    Analyze price statistics for each brand within a product.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset
    product_name : str
        Product to analyze
    price_col : str
        Price column name (default: 'Preco')
    iqr_threshold : float
        IQR multiplier for outlier detection (default: 1.5)
    
    Returns:
    --------
    pd.DataFrame : Statistics for each brand
    """
    
    product_data = df[df['Produto'] == product_name]
    
    if len(product_data) == 0:
        return pd.DataFrame()
    
    results = []
    
    for brand in product_data['Marca'].unique():
        brand_data = product_data[product_data['Marca'] == brand][price_col].dropna()
        
        q1 = brand_data.quantile(0.25)
        q3 = brand_data.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - iqr_threshold * iqr
        upper_bound = q3 + iqr_threshold * iqr
        
        outliers = brand_data[(brand_data < lower_bound) | (brand_data > upper_bound)]
        
        results.append({
            'Marca': brand,
            'count': len(brand_data),
            'mean': brand_data.mean(),
            'q1': q1,
            'median': brand_data.median(),
            'q3': q3,
            'outlier_count': len(outliers),
            'outlier_percentage': (len(outliers) / len(brand_data) * 100) if len(brand_data) > 0 else 0
        })
    
    return pd.DataFrame(results).sort_values('Marca').reset_index(drop=True)

In [29]:
# Usage example:
print("Manteiga brands analysis:")
analyze_brand_by_product(df, 'Manteiga')

Manteiga brands analysis:


,Marca,count,mean,q1,median,q3,outlier_count,outlier_percentage
0,Aviação,825,15.773491,13.9900,14.990,16.9900,19,2.303030
1,Batavo,450,12.920222,11.8900,12.590,13.2900,27,6.000000
2,Bela Nata,1,10.300000,10.3000,10.300,10.3000,0,0.000000
3,Brasileirissima,1,13.490000,13.4900,13.490,13.4900,0,0.000000
4,Carrefour,4,15.040000,10.9400,11.490,15.5900,1,25.000000
5,Catupiry,2,8.190000,7.0400,8.190,9.3400,0,0.000000
6,Claybom,6,5.946667,3.8325,6.540,7.7400,0,0.000000
7,Cruzilia,1,10.490000,10.4900,10.490,10.4900,0,0.000000
8,Da Vaca,145,13.238276,11.9900,12.990,13.9900,4,2.758621
9,Danone,467,13.249872,11.7000,12.950,14.7400,4,0.856531


In [30]:
cleaner = DataCleaner(df)
df_limpo = cleaner.clean_all(
    remove_duplicates=True,
    missing_threshold=0.5,
    outlier_method='iqr',
    encode=True,
    normalize=False
)

INICIANDO LIMPEZA COMPLETA DO DATASET
Analisando estrutura dos dados

Total de linhas: 33850
Total de colunas: 8
   - Numéricas: 3
   - Categóricas: 4
   - Data/Hora: 1
Linhas duplicadas: 1

Valores ausentes encontrados em 1 colunas:
   - PPK: 1 (0.00%)
Removidas 1 linhas duplicadas

Tratando valores ausentes...
   Preenchendo 1 colunas numéricas...

Tratando outliers (método: iqr, ação: cap)...
   Outliers tratados em 3 colunas

Codificando variáveis categóricas...
   Codificadas 4 colunas
LIMPEZA CONCLUÍDA!

Resumo das transformações:
   Linhas originais: 33850
   Linhas finais: 33849
   Colunas originais: 8
   Colunas finais: 12
   Duplicatas removidas: 1
   Colunas com outliers tratados: 3
   Colunas codificadas: 4


In [31]:
df

,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK,Classe
0,2022-09-12,SUP-0,Leite,Parmalat,6.28,1.0,6.280000,Laticínios
1,2022-09-12,SUP-1,Tomate,Sem Marca,6.99,1.0,6.990000,Vegetais
2,2022-09-12,SUP-2,Arroz,Namorado,17.98,5.0,3.596000,Grãos & Massas
3,2022-09-12,SUP-2,Arroz,Broto Legal,22.69,5.0,4.538000,Grãos & Massas
4,2022-09-12,SUP-0,Feijão,Broto Legal,10.98,1.0,NaN,Grãos & Massas
...,...,...,...,...,...,...,...,...
34027,2025-09-22,SUP-2,Manteiga,Outro,17.99,0.2,89.950000,Laticínios
34028,2025-09-22,SUP-0,Café,Caboclo,29.95,0.5,59.900000,Padaria & Cozinha
34029,2025-09-22,SUP-2,Leite,Ninho,7.99,1.0,7.990000,Laticínios
34030,2025-09-22,SUP-0,Arroz,Empório São João,29.99,5.0,5.998000,Grãos & Massas


In [32]:
df_limpo.head()

,Data_Coleta,Estabelecimento,Produto,Marca,Preco,Quantidade,PPK,Classe_Carnes Vermelhas,Classe_Grãos & Massas,Classe_Laticínios,Classe_Padaria & Cozinha,Classe_Vegetais
0,2022-09-12,0,13,88,6.28,1.00,6.280,False,False,True,False,False
1,2022-09-12,1,18,116,6.99,1.00,6.990,False,False,False,False,True
2,2022-09-12,12,0,74,17.98,1.15,3.596,False,True,False,False,False
3,2022-09-12,12,0,15,22.69,1.15,4.538,False,True,False,False,False
4,2022-09-12,0,10,15,10.98,1.00,7.596,False,True,False,False,False


In [33]:
import json

mapeamentos = cleaner.label_mappings

for coluna, mapa in mapeamentos.items():
    with open(f'./data/mapa_{coluna}.json', 'w', encoding='utf-8') as f:
        json.dump(mapa, f, ensure_ascii=False, indent=4)

print("Mapeamentos salvos com sucesso em ./data/")

Mapeamentos salvos com sucesso em ./data/


In [34]:
# Salvando os dados limpos dentro da pasta dados
df_limpo.to_excel('data/dados_limpos_ICB.xlsx', index=False)